In [ ]:
import networkx as nx
import polars as pl
Edges = pl.read_csv("/home/alberto/LPSim/traffic_phase_transition/data/carto/BOS/edges.csv")
Dt = 15
MinutesInDay = 24*60
IntDtMinutesArray = [i for i in range(0, MinutesInDay, Dt)] 
for t in IntDtMinutesArray:
    Edges[f"trafficked_{t}"] = pl.Series("", [False for _ in range(Edges.shape[0])])

In [ ]:
p = 0.5
import numpy as np
# Function to update trafficked_{t} column
Edges.group_by("u").agg([pl.col("v") ])
def update_trafficked(df, p):
    # Iterate over each index
    
    for idx in df['index']:
        # Select rows where pl.col('u') of such index is pl.col('v') of those rows
        condition = df['v'] == df['u'][idx]
        selected_rows = df.filter(condition)
        # Generate random numbers
        random_numbers = np.random.uniform(0, 1, len(selected_rows))
        
        # Update trafficked_{t} column
        for i, row in selected_rows.iterrows():
            if random_numbers[i] > p:
                df = df.with_columns(
                    pl.when(pl.col('index') == row['index'])
                    .then(pl.col('trafficked_{t}') | True)
                    .otherwise(pl.col('trafficked_{t}'))
                    .alias('trafficked_{t}')
                )
    return df


osmid,x,y,ref,highway,index
i64,f64,f64,str,str,i64
61816807,-70.844629,41.995558,null,null,0
61817002,-70.843934,41.996081,null,null,1
61817016,-70.866671,41.973369,null,null,2
61817409,-70.84401,41.995529,null,null,3
61817412,-70.872564,41.993312,null,null,4
…,…,…,…,…,…
10281222830,-71.446432,42.763661,null,null,23462
10281222831,-71.446355,42.763559,null,null,23463
10281222833,-71.44625,42.763443,null,null,23464


In [3]:
Edges

uniqueid,u,v,length,speed_mph,lanes,osmid_u,osmid_v
i64,i64,i64,f64,i64,i64,i64,i64
0,0,9,51.8,35,1,61816807,61819222
1,0,3,55.4,25,1,61816807,61817409
2,0,6,2223.4,35,1,61816807,61818775
3,1,10,1116.2,35,1,61817002,61819564
4,1,9,30.0,35,1,61817002,61819222
…,…,…,…,…,…,…,…
54946,23464,23463,15.6,20,1,10281222833,10281222831
54947,23465,23463,105.5,20,1,10281222839,10281222831
54948,23466,23186,16.9,25,1,10844948681,72474574
